In [ ]:
import holoviews as hv; hv.extension('bokeh', 'plotly', logo=None)
import panel as pn;     pn.extension()
hv.opts.defaults(hv.opts.Raster(cmap='gray', xaxis=None, yaxis=None, frame_width=300, aspect='equal'))

import numpy as np

In [ ]:
t = np.arange(-3,3,0.01)

Utrue = np.array([np.cos(17*t) * np.exp(-t**2), np.sin(11*t)]).T
Strue = np.array([[2, 0],[0, 0.5]])
Vtrue = np.array([np.sin(5*t) * np.exp(-t**2), np.cos(13*t)]).T

X = Utrue @ Strue @ Vtrue.T
h_orig=hv.Raster( X ).opts( title="Original")
#h_orig

In [ ]:
sigma = 1
Xnoisy = X + sigma*np.random.randn(*X.shape)
h_noisy=hv.Raster( Xnoisy ).opts( title="Original+Noise")
#h_noisy

In [ ]:
U, S, VT = np.linalg.svd(Xnoisy,full_matrices=0)
N        = Xnoisy.shape[0]
cutoff   = (4/np.sqrt(3)) * np.sqrt(N) * sigma # Hard threshold
r        = np.max(np.where(S > cutoff)) # Keep modes w/ sig > cutoff
print( r, cutoff, S[0:r+1])

Xclean = U[:,:(r+1)] @ np.diag(S[:(r+1)]) @ VT[:(r+1),:]
h_clean=hv.Raster( Xclean ).opts( title=f"{100.*cutoff/S[0]:.0f}% best cutoff")
#h_clean

In [ ]:
cdS = np.cumsum(S) / np.sum(S) # Cumulative energy
r90 = np.min(np.where(cdS > 0.90)) # Find r to capture 90% energy

X90 = U[:,:(r90+1)] @ np.diag(S[:(r90+1)]) @ VT[:(r90+1),:]
h_90=hv.Raster( X90 ).opts( title="90% cutoff")
#h_90

In [ ]:
(h_orig+h_noisy+h_clean+h_90).cols(2)

In [ ]:
h_s= hv.Curve(S).opts(title='Singular Values').opts(logy=True, tools=['hover'], show_grid=True, padding=0.05)*\
     hv.Scatter(S).opts(size=3)*\
     hv.Scatter(S[0:+(r+1)]).opts(color='red', size=4)*\
     hv.HLine( cutoff ).opts(color='red', line_width=.8)
     #hv.Path( np.array([[-5,-5,100,100,-5],[200,100, 100, 200, 200]]).T ).opts(line_dash='dotted', color='b')*\
h_cs = hv.Curve( cdS ).opts( title='Cumulative Single Values', show_grid=True, tools=['hover'], padding=0.05)*\
       hv.Scatter( cdS[0:r+1]).opts(size=4,color='red')*\
       hv.VLine( r90 ).opts(color='blue', line_width=0.8)*\
       hv.HLine( 0.9 ).opts(color='blue', line_width=0.8)

(h_s.opts(width=500)+h_cs.opts(width=300)).opts(shared_axes=False)